In [1]:
import pandas as pd 
import sqlite3


In [2]:
con = sqlite3.connect('database.sqlite')

filtered_data = pd.read_sql_query(" SELECT * FROM Reviews WHERE Score !=3",con)

In [3]:
filtered_data.shape

(525814, 10)

In [4]:
filtered_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [ ]:
#df = pd.read_csv('Reviews.csv')
#df.head()

In [ ]:
#data.iloc[:,:9].head().ndim

In [ ]:
#data['Text'].ndim

In [ ]:
#Sorting data according to ProductId in ascending order
#sorted_data=data.sort_values('ProductId') #, axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

In [ ]:
#Deduplication of entries
#final=sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"})#, keep='first', inplace=False)
#final.shape

In [ ]:
# How much data still remains
#(final['Id'].size*1.0)/(data['Id'].size*1.0)*100

In [ ]:
#final['Id'].size

In [ ]:
#final=final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]

In [ ]:
#final.shape

In [ ]:
#Before starting the next phase of preprocessing lets see the number of entries left
#print(final.shape)

#How many positive and negative reviews are present in our dataset?
#final['Score'].value_counts()

In [5]:
import numpy as np
import re

In [6]:
#final = np.array(final)

In [7]:
def partition(x):
    if x < 3:
        return 0
    return 1
actualScore = filtered_data['Score']
positiveNegative = actualScore.map(partition)
filtered_data['Score'] = positiveNegative
print ('Number of data points in our data',filtered_data.shape)

Number of data points in our data (525814, 10)


In [8]:
filtered_data.Score.value_counts()

1    443777
0     82037
Name: Score, dtype: int64

In [9]:
filtered_data.UserId.value_counts().head()


A3OXHLG6DIBRW8    425
A1YUL9PCJR3JTY    391
AY12DBB0U420B     365
A281NPSIMI1C2R    346
A1Z54EM24Y40LL    230
Name: UserId, dtype: int64

In [10]:
filtered_data[filtered_data.UserId=='ARSJBUI46CURR'].head()


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text


In [ ]:
#display = pd.read_sql_query(""" SELECT * FROM Reviews WHERE Score !=3 AND UserId ==ARSJBUI46CURR Or """,con)

In [12]:
#Sorting data according to ProductId in ascending order
sorted_data=filtered_data.sort_values('ProductId') #, axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

In [13]:
#Deduplication of entries
final=sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"})#, keep='first', inplace=False)
final.shape

(364173, 10)

In [15]:
# How much data still remains
(final['Id'].size*1.0)/(filtered_data['Id'].size*1.0)*100

69.25890143662969

In [ ]:
## HelpfulnessNumerator <=  HelpfulnessDenominator if not then remove such rows

In [16]:
final=final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]

In [17]:
final.shape

(364171, 10)

In [18]:
final['Score'].value_counts()

1    307061
0     57110
Name: Score, dtype: int64

## Text Processing : Stemming, Lematization,Stop words removal
### begin by removing the html tags
### Remove any punctuation or limited set of special characters like, or # etc.
### Check if the word is made up of english letters and is not alpha-numereic
### Check to see if the length of the word is greater than 2(as it was researched that there is no adjective in 2-letters)
### Convert the word to lower case
### Remove stop words
### Finally Snowball stemming the word (it was observed to be better than PorterScanning)

In [19]:
import re

In [20]:
# Removing the html tags
i = 0
for sent in final['Text'].values:
    if(len(re.findall('<.*?>',sent))):
        print(i)
        print(sent)
        break
    i += 1

6
I set aside at least an hour each day to read to my son (3 y/o). At this point, I consider myself a connoisseur of children's books and this is one of the best. Santa Clause put this under the tree. Since then, we've read it perpetually and he loves it.<br /><br />First, this book taught him the months of the year.<br /><br />Second, it's a pleasure to read. Well suited to 1.5 y/o old to 4+.<br /><br />Very few children's books are worth owning. Most should be borrowed from the library. This book, however, deserves a permanent spot on your shelf. Sendak's best.


In [22]:
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer,SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
stop =  set(stopwords.words('english')) # set of stopwords
sno = nltk.stem.SnowballStemmer('english')
def cleanhtml(sentence):
    cleantext = re.sub(r'<.*?>','',sentence)
    return cleantext

def cleanpunc(sentence): #function to clean the word of any punctuation or special character
    cleaned = re.sub(r'[?|!\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/|]',r'',cleaned)
    return cleaned
print('Stop words are:\n',stop)

Stop words are:
 {'yours', "it's", 'has', "shouldn't", 'i', 'wasn', 'now', 'each', 'did', 'ma', "needn't", 'its', 'hasn', 'ourselves', 'myself', 'you', 'same', 'y', 'between', 'being', 'up', 'theirs', 'yourselves', 'until', 'was', "isn't", 'does', 'other', "shan't", 'just', 'mustn', 'but', 'her', 'after', 'further', 'before', 'who', 'hers', 've', 'hadn', "wasn't", 'o', 'yourself', "that'll", 'she', 'they', 'too', 'what', 'there', 'when', 'the', 'herself', 'at', "won't", 'that', 'm', 'been', "mustn't", 'during', 'once', 'don', 're', 'having', 'll', 'this', 'or', 'from', "mightn't", 'have', 'why', 'can', 'not', 'which', 'own', 'nor', 'doesn', 'ours', 'weren', 'had', 'below', 'itself', "doesn't", 'while', 'if', "hasn't", 'as', 'them', 'and', 'their', 'whom', "you're", 'again', 'an', "she's", 'how', 'couldn', 'am', 'any', 'on', 'than', "didn't", "aren't", 'won', 'very', 'because', 'ain', 'such', 'should', 'by', 'me', 'against', 'under', "should've", 'both', 'himself', 'about', 'those', 'th

In [44]:
i=0
str1=''
final_string=[]
all_pos_words=[]
all_neg_words=[]
s=''

for sent in final['Text'].values:
    filtered_sentences = []
    
    #remove html tags
    sent = cleanhtml(sent)
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if((cleaned_words.isalpha())&(len(cleaned_words)>2)
                if(cleaned_words.lower() not in stop)
                    (s=sno.stem(cleaned_words.lower())
                    (filtered_sentences.append(s)
                    if(final['Score'].values[i]==1
                      all_pos_words.append(s)
                    if(final['Score'].values[i]==0:
                      all_neg_words.append(s)
                else:
                    continue
            else:
                continue
    str1=''.join(filtered_sentences)
                       
    final_string.append(str1)
    i += 1
                    

SyntaxError: invalid syntax (<ipython-input-44-a3106039792d>, line 20)

In [ ]:
final['CleanedText'] = final_string

In [ ]:
final.to_csv('Cleaned_text.csv',index=False)


In [ ]:
final=pd.read_csv('Cleaned_text.csv')
final.head()